# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27788, 27789, 27790, 27791]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
CORRELATOR
|Tesla deliveries Q3 2024|27788|Will Tesla increase its deliveries in Q3 2024 compared with Q2 2024?|
|Magnificent Seven bubble|27789|Will the bubble in the Magnificent Seven pop before September 30, 2024?|
|OpenAI GPT-5 announcement|27790|Will OpenAI announce GPT-5 before September 30, 2024?|
|Cannabis Schedule I status|27791|Will cannabis be removed from Schedule I of the Controlled Substance Act before September 30, 2024?|
NEWS
ASKING
Researching Will cannabis be removed from Schedule I of the Controlled Substance Act before September 30, 2024?
ALL
The Drug Enforcement Agency's proposal to reschedule cannabis from Schedule I to Schedule III has been met with skepticism from local marijuana professionals. Dylan Mattole, a cannabis cultivator, believes that rescheduling is an incremental improvement, but does not address the major issue of marijuana remaining illegal at the federal level. He suggests that de-scheduling would be a more effective solution. Ross G

In [9]:
fcst.report()

27788 Will Tesla increase its deliveries in Q3 2024 compared with Q2 2024? 
Forecast 65
Rationale Given the patterns in Tesla's deliveries and the recent uptick in metrics such as registrations in China, it's reasonable to believe deliveries will ascend in Q3 2024, albeit moderately. Recent quarters, despite economic and logistical hurdles, have shown a trend towards recovery and growth, making a slight increase likely. 

27789 Will the bubble in the Magnificent Seven pop before September 30, 2024?

Forecast 30
Rationale Based on current market conditions and ongoing expert discussions about a potential correction or bubble in AI-related stocks, it appears that volatility is likely. However, a 50% drop in at least four of the Magnificent Seven stocks is an extreme event. Given the companies' strong financial positions and their pivotal role in the market, a collapse of that magnitude seems unlikely in the short term, unless driven by an unprecedented negative catalyst. Thus, the probab

In [10]:
fcst.upload()

Prediction posted for  27788
Comment posted for  27788
Prediction posted for  27789
Comment posted for  27789
Prediction posted for  27790
Comment posted for  27790
Prediction posted for  27791
Comment posted for  27791
